In [1]:
import csv

file_name = 'data/WHONDRS_S19S_Sediment_GrainSize.csv'

# checking if sample_id (line[1]) is the unique column

with open(file_name) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    sample_id_set = set()
    for line in csv_reader:
        if line_count != 0:
            sample_id_set.add(line[1])
        line_count += 1

    print(f'{len(sample_id_set)} distinct samples') 
    # 276 distinct samples
    # Confirms that sample_id is the unique identifier for each row
    

276 distinct samples


In [2]:
import pandas as pd

df = pd.read_csv (file_name, index_col='Sample_ID')

print(df.isnull().values.any()) # False
# confirms that there are no nulls in the data

df = pd.read_csv (file_name, index_col='Sample_ID', na_values=['-9999'])
print(df.isnull().values.any()) # True
# confirms that -9990 are treated as null

print(df.loc['S19S_0026_BULK-M']) # Checking whether the row containing -9999s is treated with NaN values

False
True
Study_Code             WHONDRS_S19S
Percent_Fine_Sand               5.4
Percent_Med_Sand                NaN
Percent_Coarse_Sand            81.6
Percent_Tot_Sand                NaN
Percent_Clay                    5.7
Percent_Silt                    NaN
Name: S19S_0026_BULK-M, dtype: object


In [3]:
import pymongo
from pymongo import MongoClient

with open("secret.txt") as secret_file:
    uri = secret_file.readline()

client = MongoClient(uri)

db = client["berkeley"]
collection = db["sediment_grain_size"]

df.reset_index(inplace=True)
data_dict = df.to_dict("records")
collection.insert_many(data_dict)

In [4]:
data_from_db = collection.find_one({"Sample_ID": "S19S_0004_BULK-D"})
df = pd.DataFrame(list(data_from_db))
print(data_from_db)

{'_id': ObjectId('643f0933da1bf97fdf649fa4'), 'Sample_ID': 'S19S_0004_BULK-D', 'Study_Code': 'WHONDRS_S19S', 'Percent_Fine_Sand': 20.1, 'Percent_Med_Sand': 34.1, 'Percent_Coarse_Sand': 26.8, 'Percent_Tot_Sand': 81.0, 'Percent_Clay': 0.0, 'Percent_Silt': 19.0}
